In [1]:
from sklearn.metrics import fbeta_score, make_scorer, ConfusionMatrixDisplay, classification_report, confusion_matrix
from utils.data_loader import load_split, prepare_features_target
import optuna
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
from utils.learning_curve import learning_curve_with_resampling
from models.manage_models import save_model

c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
f2_scorer = make_scorer(fbeta_score, beta=2)

In [3]:
train_df = load_split('train', data_dir='dataset/splits')
X_train, y_train = prepare_features_target(train_df, target_col='Fault')

In [4]:
def objective(trial, X, y):
    ratio = float(y.value_counts()[0] / y.value_counts()[1])

    param = {
    "verbosity": 0,
    "objective": "binary:logistic",
    "n_estimators": 1000, # Subimos el techo, el early stopping hará el resto
    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
    "max_depth": trial.suggest_int("max_depth", 3, 10),
    
    # --- Nuevos parámetros de regularización ---
    "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
    "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
    "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
    "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
    # --------------------------------------------

    "subsample": trial.suggest_float("subsample", 0.6, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
    "tree_method": "hist",
    "scale_pos_weight": trial.suggest_float("scale_pos_weight", ratio * 0.9, ratio * 1.1),
    "early_stopping_rounds": 50, # Un poco más de margen para encontrar el óptimo
    "eval_metric": "logloss",
    "n_jobs": -1
}

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f2_scores = []

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = xgb.XGBClassifier(**param)
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )

        preds = model.predict(X_val)
        score = fbeta_score(y_val, preds, beta=2)
        f2_scores.append(score)

    return np.mean(f2_scores)

In [5]:
study = optuna.create_study(direction="maximize", study_name='xgboost_opt')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100, show_progress_bar=True)

[I 2025-12-22 15:04:19,064] A new study created in memory with name: xgboost_opt
Best trial: 0. Best value: 0.831734:   1%|          | 1/100 [00:17<29:25, 17.84s/it]

[I 2025-12-22 15:04:36,899] Trial 0 finished with value: 0.831733660454919 and parameters: {'learning_rate': 0.014400562479368634, 'max_depth': 9, 'min_child_weight': 6, 'gamma': 3.0329490305234676e-06, 'lambda': 0.5364617649677998, 'alpha': 0.38772275568378795, 'subsample': 0.8525534144908764, 'colsample_bytree': 0.6812603381647075, 'scale_pos_weight': 2.168972464283565}. Best is trial 0 with value: 0.831733660454919.


Best trial: 1. Best value: 0.835509:   2%|▏         | 2/100 [00:41<34:32, 21.15s/it]

[I 2025-12-22 15:05:00,362] Trial 1 finished with value: 0.8355088232677401 and parameters: {'learning_rate': 0.02541002994776132, 'max_depth': 10, 'min_child_weight': 3, 'gamma': 1.09184511900687e-06, 'lambda': 3.9230816691786647, 'alpha': 7.45434051463428e-06, 'subsample': 0.7181991641105008, 'colsample_bytree': 0.7242311506042632, 'scale_pos_weight': 2.0267778601570683}. Best is trial 1 with value: 0.8355088232677401.


Best trial: 2. Best value: 0.838605:   3%|▎         | 3/100 [00:50<25:36, 15.84s/it]

...

[I 2025-12-22 15:28:38,003] Trial 97 finished with value: 0.8468439676397852 and parameters: {'learning_rate': 0.05102075013690474, 'max_depth': 7, 'min_child_weight': 20, 'gamma': 1.078397451772485e-08, 'lambda': 2.4387555534224554e-05, 'alpha': 5.648908915512607e-07, 'subsample': 0.9650891294621274, 'colsample_bytree': 0.9665797651220802, 'scale_pos_weight': 2.3813322674293076}. Best is trial 74 with value: 0.8488555046107453.


Best trial: 74. Best value: 0.848856:  99%|█████████▉| 99/100 [24:31<00:13, 13.15s/it]

[I 2025-12-22 15:28:50,634] Trial 98 finished with value: 0.8459470088066988 and parameters: {'learning_rate': 0.05284455659867913, 'max_depth': 7, 'min_child_weight': 20, 'gamma': 1.6449213826489347e-08, 'lambda': 4.825896284058178e-05, 'alpha': 2.0431456017485656e-07, 'subsample': 0.9989058563011087, 'colsample_bytree': 0.8013559077862867, 'scale_pos_weight': 2.3966752173157655}. Best is trial 74 with value: 0.8488555046107453.


Best trial: 74. Best value: 0.848856: 100%|██████████| 100/100 [24:44<00:00, 14.84s/it]

[I 2025-12-22 15:29:03,239] Trial 99 finished with value: 0.8466124187624606 and parameters: {'learning_rate': 0.04777900070641823, 'max_depth': 7, 'min_child_weight': 18, 'gamma': 2.6574282943556776e-08, 'lambda': 0.0006087573712302937, 'alpha': 1.8566311930712078e-06, 'subsample': 0.9740799503524757, 'colsample_bytree': 0.9729903633223603, 'scale_pos_weight': 2.4129594694342735}. Best is trial 74 with value: 0.8488555046107453.
